In [1]:
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote

# 1. URL 파라미터 분리하기.
# Service URL
xmlUrl = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'

My_API_Key = unquote('%2FargzrCJK5%2BwZ0DhHr2rbJYbgS%2Bgrj9W2jtM45tBMXuSmZQkjpSezFTK4hUtq65ZuvcfgdpfjvKw1iqAfaDRaw%3D%3D')    # 아래 내가 받은 인증키가 안 되서 수업용 인증키 사용.
# My_API_Key = unquote('Agq7hySmyMi1FFU9kYibP%2BEnxYepQ%2FB6Dn%2Bw9lsYKVSCDjTwIdvpjmuhJrtyQrhipg3F3a4jbSq%2FLxHi%2FdUIoQ%3D%3D')    # 사용자 인증키
queryParams = '?' + urlencode(    # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것이다. 메타코드 아님.
    { 
      quote_plus('ServiceKey') : My_API_Key, 
      quote_plus('pageNo') : '1',
      quote_plus('numOfRows') : '10',
      quote_plus('startCreateDt') : '20200310',
      quote_plus('endCreateDt') : '20210315' }
)

response = requests.get(xmlUrl + queryParams).text.encode('utf-8')
xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')


In [2]:
response

t></item><item><accDefRate>1.8464615809</accDefRate><accExamCnt>589520</accExamCnt><accExamCompCnt>579920</accExamCompCnt><careCnt>2051</careCnt><clearCnt>8501</clearCnt><createDt>2020-04-24 10:11:08.08</createDt><deathCnt>240</deathCnt><decideCnt>10708</decideCnt><examCnt>9600</examCnt><resutlNegCnt>569212</resutlNegCnt><seq>116</seq><stateDt>20200424</stateDt><stateTime>00:00</stateTime><updateDt>2020-04-24 10:11:08.08</updateDt></item><item><accDefRate>1.8650057856</accDefRate><accExamCnt>583971</accExamCnt><accExamCompCnt>573832</accExamCompCnt><careCnt>2179</careCnt><clearCnt>8411</clearCnt><createDt>2020-04-23 10:11:05.05</createDt><deathCnt>240</deathCnt><decideCnt>10702</decideCnt><examCnt>10139</examCnt><resutlNegCnt>563130</resutlNegCnt><seq>115</seq><stateDt>20200423</stateDt><stateTime>00:00</stateTime><updateDt>2020-04-23 10:11:05.05</updateDt></item><item><accDefRate>1.8899402302</accDefRate><accExamCnt>577959</accExamCnt><accExamCompCnt>565838</accExamCompCnt><careCnt>22

In [3]:
rows = xmlobj.findAll('item')

In [4]:
rows[0]

<item><accDefRate>1.4755317026</accDefRate><accExamCnt>5061279</accExamCnt><accExamCompCnt>4901894</accExamCompCnt><careCnt>12827</careCnt><clearCnt>58253</clearCnt><createDt>2021-01-17 15:27:58.692</createDt><deathCnt>1249</deathCnt><decideCnt>72329</decideCnt><examCnt>159385</examCnt><resutlNegCnt>4829565</resutlNegCnt><seq>487</seq><stateDt>20210117</stateDt><stateTime>00:00</stateTime><updateDt>2021-04-20 15:29:06.35</updateDt></item>

In [5]:
# 한 개 행의 모든 컬럼값을 리스트에 담아보자.
columns = rows[0].find_all()
columns

[<accDefRate>1.4755317026</accDefRate>,
 <accExamCnt>5061279</accExamCnt>,
 <accExamCompCnt>4901894</accExamCompCnt>,
 <careCnt>12827</careCnt>,
 <clearCnt>58253</clearCnt>,
 <createDt>2021-01-17 15:27:58.692</createDt>,
 <deathCnt>1249</deathCnt>,
 <decideCnt>72329</decideCnt>,
 <examCnt>159385</examCnt>,
 <resutlNegCnt>4829565</resutlNegCnt>,
 <seq>487</seq>,
 <stateDt>20210117</stateDt>,
 <stateTime>00:00</stateTime>,
 <updateDt>2021-04-20 15:29:06.35</updateDt>]

In [6]:
# 모든 행과 열의 값을 모아 매트릭스로 만들어보자.
rowList = []
nameList = []
columnList = []

rowsLen = len(rows)
for i in range(0, rowsLen):
    columns = rows[i].find_all()
    
    columnsLen = len(columns)
    for j in range(0, columnsLen):
        # 첫 번째 행 데이터 값 수집 시에만 컬럼 값을 저장한다. (어차피 rows[0], rows[1], ... 모두 컬럼헤더는 동일한 값을 가지기 때문에 매번 반복할 필요가 없다.)
        if i == 0:
            nameList.append(columns[j].name)
        # 컬럼값은 모든 행의 값을 저장해야한다.    
        eachColumn = columns[j].text
        columnList.append(eachColumn)
    rowList.append(columnList)
    columnList = []    # 다음 row의 값을 넣기 위해 비워준다. (매우 중요!!)
    
result = pd.DataFrame(rowList, columns=nameList)

In [7]:
result

,accDefRate,accExamCnt,accExamCompCnt,careCnt,clearCnt,createDt,deathCnt,decideCnt,examCnt,resutlNegCnt,seq,stateDt,stateTime,updateDt
0,1.4755317026,5061279,4901894,12827,58253,2021-01-17 15:27:58.692,1249,72329,159385,4829565,487,20210117,00:00,2021-04-20 15:29:06.35
1,1.3689198014,7084930,7013340,6578,87754,2021-03-15 09:51:09.37,1675,96007,71590,6917333,449,20210315,00:00,2021-04-20 14:47:28.958
2,1.3667877215,7066394,6996551,6551,87408,2021-03-14 09:42:57.64,1669,95628,69843,6900923,448,20210314,00:00,2021-04-20 14:47:44.94
3,1.3636674313,7046775,6978901,6877,86625,2021-03-13 09:36:41.886,1667,95169,67874,6883732,447,20210313,00:00,2021-04-20 14:48:06.227
4,1.3634807664,7012657,6943919,7274,85743,2021-03-12 09:49:58.077,1662,94679,68738,6849240,446,20210312,00:00,2021-04-20 14:48:36.524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,3.3180003364,261335,243701,7402,714,2020-03-14 10:29:11.11,72,8086,17634,235615,73,20200314,00:00,2020-03-14 10:29:11.11
367,3.4584993086,248647,230707,7470,510,2020-03-13 10:07:22.22,67,7979,17940,222728,72,20200313,00:00,2020-03-13 10:07:22.22
368,3.6217442732,234998,217271,7407,333,2020-03-12 10:37:03.03,66,7869,17727,209402,71,20200312,00:00,2020-03-12 10:37:03.03
369,3.8041745358,222395,203855,7212,288,2020-03-11 10:17:19.19,60,7755,18540,196100,70,20200311,00:00,2020-03-11 10:17:19.19


## 서울 날씨 데이터 뽑기

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [13]:
import os


plt.rc("font", family="Malgun Gothic")

# 마이너스 폰트 깨지는 거
plt.rc("axes", unicode_minus = False)

In [14]:
# retina 설정해서 글씨 조금 선명하게
%config inlineBackend.figure_format = "retina"

In [19]:
from google.colab import drive
drive.mount('/content/drive')

filename = '/content/drive/MyDrive/OBS_ASOS_DD_20210323145944 (1).csv'

encoding="euc-kr"
temp = pd.read_csv(filename, encoding="euc-kr")
temp.shape

Mounted at /content/drive


(35242, 8)

In [20]:
temp.head()

,지점,지점명,일시,평균기온(°C),최저기온(°C),최저기온 시각(hhmi),최고기온(°C),최고기온 시각(hhmi)
0,90,속초,2020-03-10,7.6,5.0,2205.0,10.1,250.0
1,90,속초,2020-03-11,6.3,2.2,518.0,10.5,1449.0
2,90,속초,2020-03-12,10.8,4.1,217.0,17.0,1526.0
3,90,속초,2020-03-13,6.8,0.7,2255.0,11.9,1011.0
4,90,속초,2020-03-14,5.6,-0.6,532.0,11.7,1427.0


In [21]:
ex_type = temp['지점명'] == '서울'
df_seoul = temp[ex_type]
df_seoul

,지점,지점명,일시,평균기온(°C),최저기온(°C),최저기온 시각(hhmi),최고기온(°C),최고기온 시각(hhmi)
4081,108,서울,2020-03-10,6.4,3.2,2343.0,10.8,1.0
4082,108,서울,2020-03-11,3.8,0.0,658.0,8.3,1454.0
4083,108,서울,2020-03-12,6.8,0.2,638.0,13.3,1435.0
4084,108,서울,2020-03-13,4.9,-0.1,2400.0,8.7,1427.0
4085,108,서울,2020-03-14,3.6,-2.7,642.0,8.6,1440.0
...,...,...,...,...,...,...,...,...
4447,108,서울,2021-03-11,10.2,4.7,720.0,17.1,1436.0
4448,108,서울,2021-03-12,10.5,7.2,2353.0,14.9,1240.0
4449,108,서울,2021-03-13,9.2,4.6,650.0,14.2,1436.0
4450,108,서울,2021-03-14,9.1,4.9,701.0,14.7,1541.0


In [22]:
temp_seoul = df_seoul['평균기온(°C)'].values
temp_seoul = temp_seoul[::-1]
temp_seoul

array([ 10.6,   9.1,   9.2,  10.5,  10.2,   8.8,   6.6,   7. ,   6.2,
         6.6,   9. ,   7.2,   4.4,   2.3,   4.7,   7.8,   9.5,   8.2,
         4.2,   2.9,   0. ,   7.8,  10.8,   8.8,   1.6,  -5.8,  -8.3,
        -5.1,   1.2,   9.5,   7.6,   6.5,   5.5,   3.4,  -0.9,  -3.1,
         3.6,   6.8,   1.4,  -3.6,  -3.2,  -5.6,   5. ,   3.2,  -1.6,
        -8.8,  -2.6,   1.7,   5.3,   7.9,   7.3,   7.6,   5.5,   4.5,
        -0.1,  -6.3,  -3.8,  -5.5,  -4.9,   4.5,   1.7,   2. ,  -3.9,
        -6.7,  -7.7, -12.2, -14.9, -14.5,  -7.4,  -5.5,  -3.5,  -5.6,
        -5. ,  -4.2,  -8.9, -10.9,   0.1,   6.7,   5.8,   2.2,  -0.8,
         2.6,   4.5,   2. ,  -1. ,  -3.7,  -5.1,  -2.1,  -4.9,  -8.3,
        -8.3,  -7.7,  -1.1,   2.3,   5.3,   5.8,   1.5,  -0.5,   3.4,
         4.5,   2.1,  -0.6,   0.7,   2.4,   1.1,  -0.2,  -0.6,  -0.1,
         4.1,   5.6,   4.6,   2.9,   1.7,   5.6,   6.5,   2.8,  13.6,
        16.6,  15.9,  12.4,  11.1,  12.5,  12.2,  10.4,   9.3,   7.7,
         5.5,   8.5,

**굵은 텍스트**## 코로나 DataFrame에 서울 기온 열 합치기

In [23]:
new_df = result.assign(seoulTemp = temp_seoul)
new_df

,accDefRate,accExamCnt,accExamCompCnt,careCnt,clearCnt,createDt,deathCnt,decideCnt,examCnt,resutlNegCnt,seq,stateDt,stateTime,updateDt,seoulTemp
0,1.3690604347,7084940,7013350,6588,87754,2021-03-15 09:51:09.37,1675,96017,71590,6917333,449,20210315,00:00,null,10.6
1,1.3668864032,7066401,6996558,6558,87408,2021-03-14 09:42:57.64,1669,95635,69843,6900923,448,20210314,00:00,null,9.1
2,1.3637663657,7046782,6978908,6884,86625,2021-03-13 09:36:41.886,1667,95176,67874,6883732,447,20210313,00:00,null,9.2
3,1.3635801994,7012664,6943926,7281,85743,2021-03-12 09:49:58.077,1662,94686,68738,6849240,446,20210312,00:00,null,10.5
4,1.3634619468,6976985,6908737,7871,84675,2021-03-11 09:39:02.159,1652,94198,68248,6814539,445,20210311,00:00,null,10.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,3.3180003364,261335,243701,7402,714,2020-03-14 10:29:11.11,72,8086,17634,235615,73,20200314,00:00,2020-03-14 10:29:11.11,3.6
367,3.4584993086,248647,230707,7470,510,2020-03-13 10:07:22.22,67,7979,17940,222728,72,20200313,00:00,2020-03-13 10:07:22.22,4.9
368,3.6217442732,234998,217271,7407,333,2020-03-12 10:37:03.03,66,7869,17727,209402,71,20200312,00:00,2020-03-12 10:37:03.03,6.8
369,3.8041745358,222395,203855,7212,288,2020-03-11 10:17:19.19,60,7755,18540,196100,70,20200311,00:00,2020-03-11 10:17:19.19,3.8


In [49]:
dailyCnt = []
for i in range(370):
    dc = int(new_df.iloc[i]['decideCnt']) - int(new_df.iloc[i+1]['decideCnt'])
    dailyCnt.append(dc)
    print(dc)

first= dailyCnt[0]
dailyCnt.insert(0,first)

382
459
490
488
465
470
446
346
416
418
398
424
444
344
355
355
415
387
395
440
356
332
416
448
561
621
621
457
343
326
362
403
504
444
303
288
371
393
370
451
467
335
305
355
453
469
497
559
349
437
392
431
345
399
403
385
119
790
580
512
524
554
536
451
664
641
674
868
838
714
1020
657
820
1028
967
1050
1044
807
970
1132
1240
985
1090
868
925
1098
1046
1064
1014
1075
880
718
1030
950
689
1351
-671
1263
615
631
577
628
540
511
451
438
388
504
555
581
382
349
271
330
386
363
343
313
230
222
208
205
191
143
146
100
126
143
89
145
126
117
75
97
124
127
113
125
103
88
119
61
77
155
121
89
58
76
91
73
47
110
73
102
98
58
72
54
69
114
75
73
64
75
63
77
113
38
50
95
61
114
125
110
61
70
82
110
126
153
113
106
109
121
136
176
155
156
136
119
167
168
198
195
267
235
248
299
323
371
441
320
280
266
396
332
324
288
297
246
197
279
166
103
56
54
34
28
36
43
20
43
33
34
23
30
31
36
18
48
28
25
58
113
41
59
63
45
26
34
39
60
61
39
33
62
44
35
45
50
62
44
48
61
63
61
54
51
42
42
62
51
39
28
51
46
17

In [51]:
dc_col = result['decideCnt']
for i in range(len(dc_col)-1):
  if dc_col[i] < dc_col[i+1]:
    temp = dc_col[i+1]
    dc_col[i] = dc_col[i+1]
    dc_col[i+1] = temp

result

,accDefRate,accExamCnt,accExamCompCnt,careCnt,clearCnt,createDt,deathCnt,decideCnt,examCnt,resutlNegCnt,seq,stateDt,stateTime,updateDt
0,1.3690604347,7084940,7013350,6588,87754,2021-03-15 09:51:09.37,1675,96017,71590,6917333,449,20210315,00:00,null
1,1.3668864032,7066401,6996558,6558,87408,2021-03-14 09:42:57.64,1669,95635,69843,6900923,448,20210314,00:00,null
2,1.3637663657,7046782,6978908,6884,86625,2021-03-13 09:36:41.886,1667,95176,67874,6883732,447,20210313,00:00,null
3,1.3635801994,7012664,6943926,7281,85743,2021-03-12 09:49:58.077,1662,94686,68738,6849240,446,20210312,00:00,null
4,1.3634619468,6976985,6908737,7871,84675,2021-03-11 09:39:02.159,1652,94198,68248,6814539,445,20210311,00:00,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,3.3180003364,261335,243701,7402,714,2020-03-14 10:29:11.11,72,8086,17634,235615,73,20200314,00:00,2020-03-14 10:29:11.11
367,3.4584993086,248647,230707,7470,510,2020-03-13 10:07:22.22,67,7979,17940,222728,72,20200313,00:00,2020-03-13 10:07:22.22
368,3.6217442732,234998,217271,7407,333,2020-03-12 10:37:03.03,66,7869,17727,209402,71,20200312,00:00,2020-03-12 10:37:03.03
369,3.8041745358,222395,203855,7212,288,2020-03-11 10:17:19.19,60,7755,18540,196100,70,20200311,00:00,2020-03-11 10:17:19.19


In [40]:
for i in range(len(result['decideCnt'])-1):
  prev = result.iloc[i]['decideCnt']
  after = result.iloc[i]['decideCnt']
  if after < prev :
    print(prev,after)
  result['decideCnt'] = result['decideCnt'].replace(['39417','38746'],['38746','30417'])
result

,accDefRate,accExamCnt,accExamCompCnt,careCnt,clearCnt,createDt,deathCnt,decideCnt,examCnt,resutlNegCnt,seq,stateDt,stateTime,updateDt
0,1.3690604347,7084940,7013350,6588,87754,2021-03-15 09:51:09.37,1675,96017,71590,6917333,449,20210315,00:00,null
1,1.3668864032,7066401,6996558,6558,87408,2021-03-14 09:42:57.64,1669,95635,69843,6900923,448,20210314,00:00,null
2,1.3637663657,7046782,6978908,6884,86625,2021-03-13 09:36:41.886,1667,95176,67874,6883732,447,20210313,00:00,null
3,1.3635801994,7012664,6943926,7281,85743,2021-03-12 09:49:58.077,1662,94686,68738,6849240,446,20210312,00:00,null
4,1.3634619468,6976985,6908737,7871,84675,2021-03-11 09:39:02.159,1652,94198,68248,6814539,445,20210311,00:00,null
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366,3.3180003364,261335,243701,7402,714,2020-03-14 10:29:11.11,72,8086,17634,235615,73,20200314,00:00,2020-03-14 10:29:11.11
367,3.4584993086,248647,230707,7470,510,2020-03-13 10:07:22.22,67,7979,17940,222728,72,20200313,00:00,2020-03-13 10:07:22.22
368,3.6217442732,234998,217271,7407,333,2020-03-12 10:37:03.03,66,7869,17727,209402,71,20200312,00:00,2020-03-12 10:37:03.03
369,3.8041745358,222395,203855,7212,288,2020-03-11 10:17:19.19,60,7755,18540,196100,70,20200311,00:00,2020-03-11 10:17:19.19


## 코로나 확진자 수와 기온 상관관계 분석 (회귀)